In [19]:
from visualization.overspeed import calculate_speeds
import pandas as pd
import folium
import numpy as np

In [20]:
LINE = '182'
BRIGADE = '1'
HOUR = 17

In [21]:
localizations = pd.read_json(f'data/buses-{HOUR}.json')
localizations = localizations[(localizations['Lines'] == LINE) & (localizations['Brigade'] == BRIGADE)]

localizations = localizations.sort_values(by='Time')
localizations = localizations.drop_duplicates('Time')

In [22]:
localizations = calculate_speeds(localizations)

In [23]:
tresholds_to_color = {
    80: '#8B0000',
    50: 'red',
    30: 'orange'
}

In [24]:
def get_color(x):
    for treshold, color in tresholds_to_color.items():
        if x > treshold:
            return color
    return 'green'

In [25]:
localizations['Color'] = localizations['Speed'].apply(get_color)

In [26]:
points = np.array(localizations[['Lat', 'Lon']])

colors = localizations['Color'].to_numpy()

m = folium.Map(location=[np.mean(points[:, 0]), np.mean(points[:, 1])], zoom_start=13)

for i in range(1, len(points)):
    start = points[i - 1]
    end = points[i]
    folium.PolyLine([start, end], color=colors[i]).add_to(m)


In [29]:
legend_html = f"""
    <div style="position: fixed; bottom: 50px; right: 50px; z-index: 1000; background-color: white; padding: 10px; border: 2px solid black; border-radius: 5px;">
        <p><strong>Legend</strong></p>
        {"".join([f'<p><span style="color: {color}">■</span> speed over {treshold} km/h</p>' for treshold, color in tresholds_to_color.items()])}
        <p><span style="color: green">■</span> speed under 30 km/h</p>
    </div>
"""

legend = folium.Element(legend_html)
m.get_root().html.add_child(legend)

m